In [1]:
!nvidia-smi

Wed May 10 06:28:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
!pwd

/content


In [4]:
import torch

torch.__version__

'2.0.0+cu118'

In [5]:
from google.colab import drive
import os

root_dir = '/content/gdrive'
drive.mount(root_dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
!ls /content/gdrive/MyDrive

'Attention is All Your Need.ipynb'   langchain-ChatGLM-master
 ChatGLM-6B-main		     study_tf
'Colab Notebooks'		     Untitled0.ipynb
 data				     我的足迹
 hoho_data			     手册.gdoc
 hou2019.gdoc			     无标题文档.gdoc
 hou2019.pdf			     无标题电子表格.gsheet
 “hou2019”的已翻译副本.gdoc


In [7]:
cd /content/gdrive/MyDrive/langchain-ChatGLM-master/

/content/gdrive/MyDrive/langchain-ChatGLM-master


In [8]:
# !pip install -r requirements.txt

In [9]:
## protobuf需升级到4.22.x以上
# !pip show protobuf
# !pip uninstall protobuf
# !pip install protobuf

Name: protobuf
Version: 3.18.3
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: 3-Clause BSD License
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: google-api-core, google-cloud-bigquery, google-cloud-bigquery-storage, google-cloud-datastore, google-cloud-firestore, google-cloud-language, google-cloud-translate, googleapis-common-protos, grpcio-status, icetk, onnxruntime, proto-plus, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-metadata


In [12]:
import torch.cuda
import torch.backends

from configs import model_config
from models.chatglm_llm import ChatGLM
from chains.local_doc_qa import *
from utils import torch_gc

In [15]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.7 MB/s eta 0:00:00


In [16]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.chains.question_answering import load_qa_chain

In [17]:
reader = PdfReader('../data/d2l-zh-pytorch.pdf')

In [18]:
raw_text = ""
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [19]:
raw_text[:100]

'动⼿学深度学习\nRelease2.0.0\nAstonZhang,ZacharyC.Lipton,MuLi,andAlexanderJ.Smola\nFeb09,2023⽬录\n前⾔ 1\n安装 9\n符号 1'

In [20]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = model_config.CHUNK_SIZE,
    chunk_overlap = 100,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [21]:
len(texts)

2062

In [22]:
texts[200]

'只需尽快识别出模式并模仿他们的⾏为即可。\n相反，如果⼯作没有⼗分具体的⽬标，就需要“⾃发”地去学习了。⽐如，⽼板可能会给我们⼀⼤堆数据，然\n后要求⽤它做⼀些数据科学研究，却没有对结果有要求。这类数据中不含有“⽬标”的机器学习问题通常被\n为⽆监督学习（ unsupervised learning ） ，本书后⾯的章节将讨论⽆监督学习技术。那么⽆监督学习可以回\n答什么样的问题呢？来看看下⾯的例⼦。\n•聚类（ clustering ）问题：没有标签的情况下，我们是否能给数据分类呢？⽐如，给定⼀组照⽚，我们\n能把它们分成⻛景照⽚、狗、婴⼉、猫和⼭峰的照⽚吗？同样，给定⼀组⽤⼾的⽹⻚浏览记录，我们能\n否将具有相似⾏为的⽤⼾聚类呢？\n•主成分分析（ principal component analysis ）问题：我们能否找到少量的参数来准确地捕捉数据的线\n性相关属性？⽐如，⼀个球的运动轨迹可以⽤球的速度、直径和质量来描述。再⽐如，裁缝们已经开发\n出了⼀⼩部分参数，这些参数相当准确地描述了⼈体的形状，以适应⾐服的需要。另⼀个例⼦：在欧⼏'

In [23]:
texts[201]

'性相关属性？⽐如，⼀个球的运动轨迹可以⽤球的速度、直径和质量来描述。再⽐如，裁缝们已经开发\n出了⼀⼩部分参数，这些参数相当准确地描述了⼈体的形状，以适应⾐服的需要。另⼀个例⼦：在欧⼏\n⾥得空间中是否存在⼀种（任意结构的）对象的表⽰，使其符号属性能够很好地匹配 ?这可以⽤来描述\n实体及其关系，例如“罗⻢” \x00“意⼤利” +“法国” =“巴黎” 。\n•因果关系（ causality ）和概率图模型（ probabilistic graphical models ）问题：我们能否描述观察到的\n许多数据的根本原因？例如，如果我们有关于房价、污染、犯罪、地理位置、教育和⼯资的⼈⼝统计数\n据，我们能否简单地根据经验数据发现它们之间的关系？\n•⽣成对抗性⽹络（ generative adversarial networks ） ：为我们提供⼀种合成数据的⽅法，甚⾄像图像和\n⾳频这样复杂的⾮结构化数据。潜在的统计机制是检查真实和虚假数据是否相同的测试，它是⽆监督\n学习的另⼀个重要⽽令⼈兴奋的领域。\n1.3.3与环境互动'

In [24]:
# protobuf貌似要升到4.22.x以上，否则下面代码会出错
embeddings = HuggingFaceEmbeddings(model_name = model_config.embedding_model_dict["ernie-base"], 
                                   model_kwargs = {'device': model_config.EMBEDDING_DEVICE})

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/nghuyong_ernie-3.0-base-zh were not used when initializing ErnieModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing ErnieModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ErnieModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
vector_store = FAISS.from_texts(texts, embeddings)
# vector_store = FAISS.from_documents(texts, embeddings)

FAISS.similarity_search_with_score_by_vector = similarity_search_with_score_by_vector
vector_store.chunk_size = model_config.CHUNK_SIZE

In [26]:
llm = ChatGLM()
llm.load_model(model_name_or_path = model_config.llm_model_dict[model_config.LLM_MODEL],
               llm_device = model_config.LLM_DEVICE,
               use_ptuning_v2 = model_config.USE_PTUNING_V2)
llm.history_len = LLM_HISTORY_LEN

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/d8a6cfc6cb4d5b0fbc51183e26c06f4a3cb26947/quantization_kernels.c
Compiling gcc -O3 -fPIC -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/d8a6cfc6cb4d5b0fbc51183e26c06f4a3cb26947/quantization_kernels.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/d8a6cfc6cb4d5b0fbc51183e26c06f4a3cb26947/quantization_kernels.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/d8a6cfc6cb4d5b0fbc51183e26c06f4a3cb26947/quantization_kernels.so
Using quantization cache
Applying quantization to glm layers


In [27]:
def answer_based_on_knowledge(query, vector_store, llm, chat_history = []):
    related_docs_with_score = vector_store.similarity_search_with_score(query, k = VECTOR_SEARCH_TOP_K)
    related_docs = get_docs_with_score(related_docs_with_score)
    prompt = generate_prompt(related_docs, query)

    # if streaming:
    #     for result, history in self.llm._stream_call(prompt = prompt,history = chat_history):
    #         history[-1][0] = query
    #         response = {"query": query,
    #                     "result": result,
    #                     "source_documents": related_docs}
    #         yield response, history
    # else:
    for result, history in llm._call(prompt = prompt, history = chat_history, streaming = True):
        history[-1][0] = query
        response = {"query": query,
                    "result": result,
                    "source_documents": related_docs}
        yield response, history

In [28]:
from IPython.display import display, Markdown, clear_output

def display_answer(query, chat_history = []):
    for resp, history in answer_based_on_knowledge(query, vector_store, llm, chat_history):
        clear_output(wait = True)
        display(Markdown(resp['result']))
    
    return resp, history

In [29]:
resp, chat_history = display_answer("Transformer的架构是怎么样的")

Transformer 是一种用于自然语言处理的深度学习模型，其架构如下：

1. Encoder 层： Transformer 使用多个Encoder 层来对输入文本进行建模。Encoder 层由多个全连接层组成，每个全连接层都使用不同的激活函数，如ReLU、Sigmoid、tanh等。

2. Decoder 层： 与Encoder 层相反，Decoder 层用于对输出文本进行建模。Decoder 层也由多个全连接层组成，每个全连接层都使用不同的激活函数，如ReLU、Sigmoid、tanh等。

3. 多头自注意力机制： Transformer 使用多头自注意力机制来捕捉输入文本中的多个维度信息，如单词、句子、段落等。每个自注意力头都会对输入文本中的不同维度信息进行加权，并计算权重结果，最后将这些加权结果组合起来得到模型的输出。

4. 残差连接： Transformer 还使用残差连接来增强模型的表达能力。残差连接是指将Encoder 层的输出与Decoder 层的输出之间的连接，这种连接可以使得模型输出更加准确地捕捉输入文本中的信息。

Transformer 的架构通过多个Encoder 层和多头自注意力机制来对输入文本进行建模，并使用残差连接来增强模型的表达能力，从而取得了在自然语言处理领域的良好表现。

In [30]:
resp, chat_history = display_answer("什么是DenseNet模型?")

DenseNet是一种循环神经网络架构，最初由Google的Yann LeCun提出，用于图像分类任务。该模型在ImageNet图像分类竞赛中取得了非常好的成绩，成为了深度学习领域的重要里程碑之一。

DenseNet模型的核心思想是将每个隐藏单元的信息传递到整个网络，使得网络能够更好地捕捉隐藏单元之间的信息。该模型中，每个隐藏单元由一个全连接层和一个激活函数组成，全连接层将输入的样本映射到隐藏状态，激活函数则用于对隐藏状态进行非线性变换。

在DenseNet模型中，每个隐藏单元都对应于图像中的一个像素，因此可以将每个像素的信息通过卷积操作和池化操作传递到整个网络。在激活函数方面，DenseNet使用ReLU，该函数具有高输入值的非线性特征，可以增强网络的表达能力。

DenseNet模型的优点是能够更好地捕捉隐藏单元之间的信息，从而提高模型的分类精度。同时，该模型的结构简单易实现，可以方便地应用于各种分类任务。然而，该模型的训练和优化需要一些技巧，例如使用小批量随机梯度下降(SGD)和残差学习率等。

In [31]:
resp

{'query': '什么是DenseNet模型?',
 'result': 'DenseNet是一种循环神经网络架构，最初由Google的Yann LeCun提出，用于图像分类任务。该模型在ImageNet图像分类竞赛中取得了非常好的成绩，成为了深度学习领域的重要里程碑之一。\n\nDenseNet模型的核心思想是将每个隐藏单元的信息传递到整个网络，使得网络能够更好地捕捉隐藏单元之间的信息。该模型中，每个隐藏单元由一个全连接层和一个激活函数组成，全连接层将输入的样本映射到隐藏状态，激活函数则用于对隐藏状态进行非线性变换。\n\n在DenseNet模型中，每个隐藏单元都对应于图像中的一个像素，因此可以将每个像素的信息通过卷积操作和池化操作传递到整个网络。在激活函数方面，DenseNet使用ReLU，该函数具有高输入值的非线性特征，可以增强网络的表达能力。\n\nDenseNet模型的优点是能够更好地捕捉隐藏单元之间的信息，从而提高模型的分类精度。同时，该模型的结构简单易实现，可以方便地应用于各种分类任务。然而，该模型的训练和优化需要一些技巧，例如使用小批量随机梯度下降(SGD)和残差学习率等。',
 'source_documents': [Document(page_content="log(^yj) =log(exp(oj\x00max (ok))∑\nkexp(ok\x00max (ok)))\n=log(exp(oj\x00max (ok)))\x00log(∑\nkexp(ok\x00max (ok)))\n=oj\x00max (ok)\x00log(∑\nkexp(ok\x00max (ok)))\n:(3.7.2)\n我们也希望保留传统的 softmax函数，以备我们需要评估通过模型输出的概率。但是，我们没有将 softmax概\n率传递到损失函数中，⽽是在交叉熵损失函数中传递未规范化的预测，并同时计算 softmax及其对数，这是\n⼀种类似 “LogSumExp 技巧”57的聪明⽅式。\nloss =nn.CrossEntropyLoss(reduction ='none ')\n3.7.3优化算法\n在这⾥，我们使⽤学习率为 0.1的⼩批量随机梯度下降作为优化算法。这与我们在线性回归例⼦中的相同，这\n说明了优化器的普适性。\ntrainer 

In [32]:
resp, chat_history = display_answer("解释一下注意力机制")

注意力机制(Attention Mechanism)是一种在神经网络中用于处理输入序列的重要技术。它允许神经网络对输入序列中的每个元素分配一个独特的权重，使得网络能够更好地捕捉输入序列中的重要信息。

注意力机制的核心思想是将输入序列中的每个元素与输出序列中的每个元素联系起来，并计算每个元素对输出序列的影响。具体来说，注意力机制会为每个元素生成一个独特的权重向量，这些权重向量表示该元素在输出序列中的重要性。然后，网络将这些权重向量拼接起来，生成一个表示输入序列和输出序列的向量。

在神经网络中，注意力机制通常被放置在输入序列和输出序列之间的过渡区域，以处理输入序列和输出序列之间的转换。例如，在语音识别任务中，注意力机制可以用于将声音序列转换为文本序列，或者将文本序列转换为声音序列。

注意力机制允许神经网络更好地捕捉输入序列中的重要信息，并使得神经网络更加智能和自适应。它在许多自然语言处理和计算机视觉任务中得到了广泛应用。

In [35]:
# 未接入知识库，未微调模型的结果
query = "Transformer的架构是怎么样的"
for result, history in llm._call(prompt = query, history = [], streaming = False):
  display(Markdown(result))

Transformer是一种用于自然语言处理的神经网络模型，其架构如下：

1. 输入层：输入为序列，如文本或语音序列。

2. 隐藏层：将输入序列映射到高维空间，通常使用多层Transformer结构。

3. Transformer头：包含一些处理输入序列的预处理操作，如编码器和解码器。

4. 编码器/解码器：编码器将输入序列映射到低维空间，并将其存储在编码器的输出中。解码器将该低维表示映射回原始序列，并将其存储在解码器的输出中。

5. 输出层：输出层的神经元数量取决于模型的规模和应用场景。

6. 输出层：输出层的神经元数量取决于模型的规模和应用场景。

在Transformer中，每个输入序列都对应着一个编码器和解码器，编码器的输出被解码器用于生成序列。编码器和解码器之间的结构可以看作是一个递归神经网络，其中每个递归层都对应着一个编码器和解码器。通过这种方式，Transformer可以处理变长的输入序列，并生成高质量的输出。Transformer在自然语言处理任务中表现良好，如机器翻译、文本分类和问答系统等。